In [ ]:
# !pip install gurobipy

In [5]:
from gurobipy import *

## 問題例13-1 CVaR

In [2]:
# nと初期配置を変更しなければいけない

O1=[1,2,3]

S=[i for i in range(1,3)]
H=[i for i in range(1,3)]
P=[i for i in range(1,4)]
T=[i for i in range(1,4)]
n_=[1,1,1]
n1=[n_[O1[0]-1],n_[O1[1]-1],n_[O1[2]-1]]

S,H,P,T,n1

([1, 2], [1, 2], [1, 2, 3], [1, 2, 3], [1, 1, 1])

In [3]:
O=[
  [1,2,3],
  [1,2,3],
  [1,2,3],
  [1,2,3],
  [1,2,3],
  [1,2,3],
  [2,1,3],
  [2,1,3],
  [2,1,3],
  [3,1,2]
]

In [4]:
m=Model("IP3")

alpha=m.addVar(vtype="C")

# 変数の定義
x,y,z,w={},{},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")


for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")


for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      z[s,h,i]=m.addVar(vtype="c",lb=0)

for i in range(len(O)):
  w[i]=m.addVar(vtype="C",lb=0)

# 制約条件
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n1[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])


for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[k,s,t+1] for s in S if s!=k)<=1)

for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      for j,p in enumerate(o):
        m.addConstr(z[s,h,i]>=quicksum(x[s,h,k,len(T)] for k in o[j:])-quicksum(x[s,h-1,k,len(T)] for k in o[j:]))

for i in range(len(O)):
  m.addConstr(w[i]>=quicksum(z[s,h,i] for s in S for h in H if h!=1)-alpha)

Restricted license - for non-production use only - expires 2023-10-25


In [5]:
# 初期配置の制約

m.addConstr(x[1,1,O1[1],1]==1)
m.addConstr(x[1,2,O1[0],1]==1)
m.addConstr(x[2,1,O1[2],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [6]:
beta=0.7
m.setObjective(alpha+10**(-5)*quicksum(y[s,k,t] for s in S for k in S if s!=k for t in range(1,len(T)))+quicksum(w[i] for i in range(len(O)))/((1-beta)*len(O)))

In [7]:
m.optimize()
print(m.Status)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 155 rows, 71 columns and 631 nonzeros
Model fingerprint: 0x25bcc5fe
Variable types: 31 continuous, 40 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.3333433
Presolve removed 151 rows and 67 columns
Presolve time: 0.03s
Presolved: 4 rows, 4 columns, 8 nonzeros
Variable types: 3 continuous, 1 integer (1 binary)

Root relaxation: cutoff, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         0.33334    0.33334  0.00%     -    0s

Explored 1 nodes (2 simplex iterations) in 0.06 seconds (0.00 

In [8]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  for (s,h,p,t) in x:
    if t==1:
      if x[s,h,p,t].X>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    if t>=2:
      if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  for (s,k,t) in y:
    if y[s,k,t].X>EPS:
      print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))

  for (s,h,i) in z:
    if z[s,h,i].X>EPS:
      print("z[%2s,%2s,%2s]=%3s" %(s,h,i,z[s,h,i].X))
  print("the objective function", m.objVal)
  print("VaR=",alpha.X)

x[ 1, 1, 2, 1]=1.0
x[ 1, 2, 1, 1]=1.0
x[ 2, 1, 3, 1]=1.0
x[ 2, 2, 1, 2]=1.0
y[ 1, 2, 1]=1.0
z[ 2, 2, 9]=1.0
the objective function 0.33334333333333327
VaR= 0.0


## 問題例13-2 CVaRの頑健性

In [9]:
# nと初期配置を変更しなければいけない

O1=[1,2,3,4]

S=[i for i in range(1,5)]
H=[i for i in range(1,5)]
P=[i for i in range(1,5)]
T=[i for i in range(1,6)]
n_=[1,2,4,3]
n1=[n_[O1[0]-1],n_[O1[1]-1],n_[O1[2]-1],n_[O1[3]-1]]

# S,H,P,T,n1,O

In [10]:
from asyncio.constants import SENDFILE_FALLBACK_READBUFFER_SIZE
import random
from random import seed
import numpy as np
from scipy.stats import multivariate_normal

np.random.seed(0)

# 期待値と分散共分散行列の準備
mean = np.array([1, 3, 5, 7])
cov = np.array([[2.5,1.2,-1.3,0.6],
               [1.2,3,1.5,-2.0],
               [-1.3,1.5,2,0.9],
               [0.6,-2.0,0.9,1.5]])

# numpy を用いた生成
size=10
data_1 = np.random.multivariate_normal(mean, cov, size=size)

# print(data_1.shape)

# print(np.mean(data_1, axis=0))

# print(np.cov(data_1, rowvar=False))

# print(data_1)
O_=np.argsort(data_1)
O_=O_+np.ones((size,4)).astype(int)
O=O_.tolist()

# data_2 = multivariate_normal(mean, cov).rvs(size=200)

# print(data_2.shape)

# print(np.mean(data_2, axis=0))

# print(np.cov(data_2, rowvar=False))
O

/tmp/ipykernel_4489/966125079.py:18: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(mean, cov, size=size)


[[2, 1, 3, 4],
 [1, 2, 3, 4],
 [2, 1, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 4, 3, 2],
 [1, 2, 3, 4],
 [2, 1, 3, 4],
 [1, 2, 4, 3],
 [2, 1, 3, 4]]

In [11]:
a=[]
d={}
for i in O:
  # print(i)
  if not i in a:
    a.append(i)
for j in a:
  d[tuple(j)]=O.count(j)

d


{(2, 1, 3, 4): 4, (1, 2, 3, 4): 4, (1, 4, 3, 2): 1, (1, 2, 4, 3): 1}

In [12]:
m=Model("IP3")

alpha=m.addVar(vtype="C")

# 変数の定義
x,y,z,w={},{},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")


for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")


for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      z[s,h,i]=m.addVar(vtype="c",lb=0)

for i in range(len(O)):
  w[i]=m.addVar(vtype="C",lb=0)

# 制約条件
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n1[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])


for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[k,s,t+1] for s in S if s!=k)<=1)

for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      for j,p in enumerate(o):
        m.addConstr(z[s,h,i]>=quicksum(x[s,h,k,len(T)] for k in o[j:])-quicksum(x[s,h-1,k,len(T)] for k in o[j:]))

for i in range(len(O)):
  m.addConstr(w[i]>=quicksum(z[s,h,i] for s in S for h in H if h!=1)-alpha)

In [13]:
m=Model("IP3")

alpha=m.addVar(vtype="C")

# 変数の定義
x,y,z,w={},{},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")


for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")


for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      z[s,h,i]=m.addVar(vtype="c",lb=0)

for i in range(len(O)):
  w[i]=m.addVar(vtype="C",lb=0)

# 制約条件
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n1[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])


for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[k,s,t+1] for s in S if s!=k)<=1)

for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      for j,p in enumerate(o):
        m.addConstr(z[s,h,i]>=quicksum(x[s,h,k,len(T)] for k in o[j:])-quicksum(x[s,h-1,k,len(T)] for k in o[j:]))

for i in range(len(O)):
  m.addConstr(w[i]>=quicksum(z[s,h,i] for s in S for h in H if h!=1)-alpha)

In [14]:
# 初期配置の制約

m.addConstr(x[1,1,O1[3],1]==1)
m.addConstr(x[1,2,O1[3],1]==1)
m.addConstr(x[1,3,O1[2],1]==1)
m.addConstr(x[2,1,O1[3],1]==1)
m.addConstr(x[2,2,O1[2],1]==1)
m.addConstr(x[2,3,O1[1],1]==1)
m.addConstr(x[3,1,O1[2],1]==1)
m.addConstr(x[3,2,O1[2],1]==1)
m.addConstr(x[4,1,O1[1],1]==1)
m.addConstr(x[4,2,O1[0],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [15]:
beta=0.8
m.setObjective(alpha+10**(-5)*quicksum(y[s,k,t] for s in S for k in S if s!=k for t in range(1,len(T)))+quicksum(w[i] for i in range(len(O)))/((1-beta)*len(O)))

In [16]:
m.optimize()
print(m.Status)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1307 rows, 499 columns and 7702 nonzeros
Model fingerprint: 0x8bb348d7
Variable types: 131 continuous, 368 integer (368 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-05, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 721 rows and 227 columns
Presolve time: 0.03s
Presolved: 586 rows, 272 columns, 3199 nonzeros
Variable types: 51 continuous, 221 integer (221 binary)
Found heuristic solution: objective 3.5000300

Root relaxation: objective 2.268206e-05, 501 iterations, 0.02 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00002    0  107    3.50003    0.00002   100%     -    0s
H    0     0              

In [17]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  for (s,h,p,t) in x:
    if t==1:
      if x[s,h,p,t].X>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    if t>=2:
      if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  for (s,k,t) in y:
    if y[s,k,t].X>EPS:
      print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))

  for (s,h,i) in z:
    if z[s,h,i].X>EPS:
      print("z[%2s,%2s,%2s]=%3s" %(s,h,i,z[s,h,i].X))
  print("the objective function", m.objVal)
  print("VaR=",alpha.X)

x[ 1, 1, 4, 1]=1.0
x[ 1, 2, 4, 1]=1.0
x[ 1, 3, 1, 3]=1.0
x[ 1, 3, 3, 1]=1.0
x[ 2, 1, 4, 1]=1.0
x[ 2, 2, 3, 1]=1.0
x[ 2, 3, 2, 1]=1.0
x[ 3, 1, 3, 1]=1.0
x[ 3, 2, 3, 1]=1.0
x[ 3, 3, 3, 2]=1.0
x[ 3, 4, 3, 5]=1.0
x[ 4, 1, 2, 1]=1.0
x[ 4, 2, 1, 1]=1.0
x[ 4, 2, 2, 4]=1.0
y[ 1, 3, 1]=1.0
y[ 2, 3, 4]=1.0
y[ 2, 4, 3]=1.0
y[ 4, 1, 2]=1.0
the objective function 4e-05
VaR= 0.0


In [18]:
O

[[2, 1, 3, 4],
 [1, 2, 3, 4],
 [2, 1, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 4, 3, 2],
 [1, 2, 3, 4],
 [2, 1, 3, 4],
 [1, 2, 4, 3],
 [2, 1, 3, 4]]

## 問題例 13-3 5×5

In [1]:
# nと初期配置を変更しなければいけない
ship_num=5
stack_num=5
height=5
time = 10

O1=[i+1 for i in range(ship_num)]

S=[i+1 for i in range(stack_num)]
H=[i+1 for i in range(height)]
P=[i+1 for i in range(ship_num)]
T=[i+1 for i in range(time)]
n1=[3,2,2,3,3]
# n1=[n_[O1[0]-1],n_[O1[1]-1],n_[O1[2]-1],n_[O1[3]-1]]

# S,H,P,T,n1,O

In [2]:
from asyncio.constants import SENDFILE_FALLBACK_READBUFFER_SIZE
import random
from random import seed
import numpy as np
from scipy.stats import multivariate_normal

np.random.seed(0)

# 期待値と分散共分散行列の準備
mean = np.arange(1,10,2)
cov=np.zeros((ship_num,ship_num))
for i in range(ship_num):
  for j in range(ship_num):
    if i==j:
      cov[i][i]=4*np.random.randn()
    elif i>j:
      cov[i][j]=np.random.randn()
      cov[j][i]=cov[i][j]

# numpy を用いた生成
size=20
data_1 = np.random.multivariate_normal(mean, cov, size=size)

# print(data_1.shape)

# print(np.mean(data_1, axis=0))

# print(np.cov(data_1, rowvar=False))

# print(data_1)
O_=np.argsort(data_1)
O_=O_+np.ones((size,ship_num)).astype(int)
O=O_.tolist()

# data_2 = multivariate_normal(mean, cov).rvs(size=200)

# print(data_2.shape)

# print(np.mean(data_2, axis=0))

# print(np.cov(data_2, rowvar=False))

/tmp/ipykernel_5235/522717571.py:22: RuntimeWarning: covariance is not positive-semidefinite.
  data_1 = np.random.multivariate_normal(mean, cov, size=size)


In [3]:
a=[]
d={}
for i in O:
  # print(i)
  if not i in a:
    a.append(i)
for j in a:
  d[tuple(j)]=O.count(j)

d


{(2, 1, 3, 5, 4): 2,
 (2, 1, 3, 4, 5): 4,
 (2, 1, 4, 3, 5): 1,
 (1, 2, 3, 5, 4): 1,
 (3, 1, 4, 2, 5): 1,
 (1, 2, 3, 4, 5): 7,
 (1, 3, 2, 4, 5): 2,
 (1, 2, 4, 3, 5): 1,
 (1, 3, 4, 2, 5): 1}

In [6]:
m=Model("IP3")

alpha=m.addVar(vtype="C")

# 変数の定義
x,y,z,w={},{},{},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")


for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")


for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      z[s,h,i]=m.addVar(vtype="c",lb=0)

for i in range(len(O)):
  w[i]=m.addVar(vtype="C",lb=0)

# 制約条件
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n1[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])


for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[k,s,t+1] for s in S if s!=k)<=1)

for s in S:
  for h in range(2,len(H)+1):
    for i,o in enumerate(O):
      for j,p in enumerate(o):
        m.addConstr(z[s,h,i]>=quicksum(x[s,h,k,len(T)] for k in o[j:])-quicksum(x[s,h-1,k,len(T)] for k in o[j:]))

for i in range(len(O)):
  m.addConstr(w[i]>=quicksum(z[s,h,i] for s in S for h in H if h!=1)-alpha)

Restricted license - for non-production use only - expires 2023-10-25


In [7]:
# 初期配置の制約

m.addConstr(x[1,1,O1[4],1]==1)
m.addConstr(x[1,2,O1[2],1]==1)
m.addConstr(x[1,3,O1[0],1]==1)
# m.addConstr(x[1,4,O1[3],1]==1)
m.addConstr(x[2,1,O1[4],1]==1)
m.addConstr(x[2,2,O1[3],1]==1)
# m.addConstr(x[2,3,O1[3],1]==1)
# m.addConstr(x[2,4,O1[1],1]==1)
m.addConstr(x[3,1,O1[3],1]==1)
m.addConstr(x[3,2,O1[3],1]==1)
m.addConstr(x[3,3,O1[0],1]==1)
# m.addConstr(x[3,4,O1[2],1]==1)
# m.addConstr(x[3,5,O1[0],1]==1)
m.addConstr(x[4,1,O1[4],1]==1)
m.addConstr(x[4,2,O1[1],1]==1)
m.addConstr(x[4,3,O1[2],1]==1)
# m.addConstr(x[4,4,O1[2],1]==1)
m.addConstr(x[5,1,O1[1],1]==1)
m.addConstr(x[5,2,O1[0],1]==1)
# m.addConstr(x[6,1,O1[7],1]==1)
# m.addConstr(x[6,2,O1[4],1]==1)
# m.addConstr(x[6,3,O1[6],1]==1)
# m.addConstr(x[6,4,O1[3],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [8]:
n1

[3, 2, 2, 3, 3]

In [9]:
beta=0.8
m.setObjective(alpha+10**(-5)*quicksum(y[s,k,t] for s in S for k in S if s!=k for t in range(1,len(T)))+quicksum(w[i] for i in range(len(O)))/((1-beta)*len(O)))

In [10]:
m.optimize()
print(m.Status)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [38]:
EPS=1.e-6

if m.Status == GRB.OPTIMAL:
  for (s,h,p,t) in x:
    if t==1:
      if x[s,h,p,t].X>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

    if t>=2:
      if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
        print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  for (s,k,t) in y:
    if y[s,k,t].X>EPS:
      print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))

  for (s,h,i) in z:
    if z[s,h,i].X>EPS:
      print("z[%2s,%2s,%2s]=%3s" %(s,h,i,z[s,h,i].X))
  print("the objective function", m.objVal)
  print("VaR=",alpha.X)

x[ 1, 1, 5, 1]=1.0
x[ 1, 2, 3, 1]=1.0
x[ 1, 3, 1, 1]=1.0
x[ 1, 3, 3, 4]=1.0
x[ 2, 1, 5, 1]=1.0
x[ 2, 2, 4, 1]=1.0
x[ 3, 1, 4, 1]=1.0
x[ 3, 2, 4, 1]=1.0
x[ 3, 3, 1, 1]=1.0
x[ 3, 4, 1, 2]=1.0
x[ 3, 5, 1, 3]=1.0
x[ 4, 1, 5, 1]=1.0
x[ 4, 2, 2, 1]=1.0
x[ 4, 3, 2, 5]=1.0
x[ 4, 3, 3, 1]=1.0
x[ 5, 1, 2, 1]=1.0
x[ 5, 1, 4, 6]=1.0
x[ 5, 2, 1, 1]=1.0
y[ 1, 3, 1]=1.0
y[ 2, 5, 5]=1.0
y[ 4, 1, 3]=1.0
y[ 5, 3, 2]=1.0
y[ 5, 4, 4]=1.0
the objective function 5.000000000000001e-05
VaR= 0.0
